In [1]:
import os
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
# import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split

from tensorflow.keras.callbacks import *
from tensorflow.keras import backend as K

# from tensorflow_addons.losses import SigmoidFocalCrossEntropy
# from tensorflow_addons.optimizers import CyclicalLearningRate

In [2]:
class CyclicLR(Callback):
    """This callback implements a cyclical learning rate policy (CLR).
    The method cycles the learning rate between two boundaries with
    some constant frequency, as detailed in this paper (https://arxiv.org/abs/1506.01186).
    The amplitude of the cycle can be scaled on a per-iteration or 
    per-cycle basis.
    This class has three built-in policies, as put forth in the paper.
    "triangular":
        A basic triangular cycle w/ no amplitude scaling.
    "triangular2":
        A basic triangular cycle that scales initial amplitude by half each cycle.
    "exp_range":
        A cycle that scales initial amplitude by gamma**(cycle iterations) at each 
        cycle iteration.
    For more detail, please see paper.
    
    # Example
        ```python
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., mode='triangular')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```
    
    Class also supports custom scaling functions:
        ```python
            clr_fn = lambda x: 0.5*(1+np.sin(x*np.pi/2.))
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., scale_fn=clr_fn,
                                scale_mode='cycle')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```    
    # Arguments
        base_lr: initial learning rate which is the
            lower boundary in the cycle.
        max_lr: upper boundary in the cycle. Functionally,
            it defines the cycle amplitude (max_lr - base_lr).
            The lr at any cycle is the sum of base_lr
            and some scaling of the amplitude; therefore 
            max_lr may not actually be reached depending on
            scaling function.
        step_size: number of training iterations per
            half cycle. Authors suggest setting step_size
            2-8 x training iterations in epoch.
        mode: one of {triangular, triangular2, exp_range}.
            Default 'triangular'.
            Values correspond to policies detailed above.
            If scale_fn is not None, this argument is ignored.
        gamma: constant in 'exp_range' scaling function:
            gamma**(cycle iterations)
        scale_fn: Custom scaling policy defined by a single
            argument lambda function, where 
            0 <= scale_fn(x) <= 1 for all x >= 0.
            mode paramater is ignored 
        scale_mode: {'cycle', 'iterations'}.
            Defines whether scale_fn is evaluated on 
            cycle number or cycle iterations (training
            iterations since start of cycle). Default is 'cycle'.
    """

    def __init__(self, base_lr=0.001, max_lr=0.006, step_size=2000., mode='triangular',
                 gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1/(2.**(x-1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma**(x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        """Resets cycle iterations.
        Optional boundary/step size adjustment.
        """
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size
        self.clr_iterations = 0.
        
    def clr(self):
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(self.clr_iterations)
        
    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())        
            
    def on_batch_end(self, epoch, logs=None):
        
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        
        K.set_value(self.model.optimizer.lr, self.clr())

In [3]:
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu_devices[0], False)

In [4]:
model_name = 'NASNet_Transfer_Mask_NOMEAN_1_CORONA'

In [5]:
home = os.path.expanduser('~')
base = os.path.join('Datasets', 'ImageCLEF', 'Coronal_Slice_Masks_300')

train_dir = os.path.join(home, base, 'train')
test_dir = os.path.join(home, base, 'test')

In [6]:
seed = 42
shuffle = True
inp_shp = (224, 224)
train_batch_size, val_batch_size = 16, 64

train_datagen = keras.preprocessing.image.ImageDataGenerator(
#                     rescale=1./255,
                    horizontal_flip=True
)

val_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=inp_shp,
        batch_size=train_batch_size,
        seed=seed,
        class_mode='categorical',
        color_mode='rgb',
        shuffle=shuffle
)

val_generator = val_datagen.flow_from_directory(
        test_dir,
        target_size=inp_shp,
        batch_size=val_batch_size,
        seed=seed,
        class_mode='categorical',
        color_mode='rgb',
        shuffle=shuffle
)

Found 121203 images belonging to 5 classes.
Found 29748 images belonging to 5 classes.


In [7]:
input_shape = (224, 224, 3)

MobileNet = tf.keras.applications.MobileNet(
    input_shape=input_shape,
    alpha=1.0,
    depth_multiplier=1,
    dropout=0.1,
    include_top=False,
    weights="imagenet"
)

MobileNet.trainable = False
MobileNet.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32

In [27]:
# CONv/FC -> BatchNorm -> ReLU(or other activation) -> Dropout -> CONV/FC -> ...

def get_model(base_model, input_shape):
    
    inputs = keras.Input(shape=input_shape)
    x = keras.applications.nasnet.preprocess_input(inputs)
    y = base_model(x, training=False)
    
    z = layers.MaxPooling2D(pool_size=(3, 3))(y)
    
    x = layers.Conv2D(256, (2, 2), activation=None)(y)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(alpha=0.1)(x)
#     x = layers.Dropout(rate=0.5)(x)
#     x = layers.MaxPooling2D(pool_size=(2, 2))(x)

    x = layers.Conv2D(128, (2, 2), activation=None)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(alpha=0.1)(x)
#     x = layers.Dropout(rate=0.5)(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    
    x = layers.Concatenate(axis=3)([x, z])
#     x = layers.Conv2D(32, (3, 3), activation=None)(x)
#     x = layers.BatchNormalization()(x)
#     x = layers.LeakyReLU(alpha=0.1)(x)
#     x = layers.Dropout(rate=0.5)(x)
#     x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    
    x = layers.Flatten()(x)
    
    x = layers.Dense(units=96, activation=None)(x)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(alpha=0.1)(x)
    y = layers.Dropout(rate=0.25)(x)

    x = layers.Dense(units=64, activation=None)(y)
    x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(alpha=0.1)(x)
    x = layers.Dropout(rate=0.25)(x)
    
    x = layers.Concatenate(axis=1)([x, y])
    
    x = layers.Dense(units=5, activation=None)(x)
    output = layers.Softmax()(x)

    model = keras.Model(inputs=inputs, outputs=output, name=f'{model_name}')
    
    return model
#     model.compile(optimizer="Adam", loss="mse", metrics=["mae"])

    

In [24]:
model = get_model(MobileNet, input_shape)
model.summary()

Model: "NASNet_Transfer_Mask_NOMEAN_1_CORONA"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
tf_op_layer_truediv_6 (TensorFl [(None, 224, 224, 3) 0           input_8[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_sub_6 (TensorFlowOp [(None, 224, 224, 3) 0           tf_op_layer_truediv_6[0][0]      
__________________________________________________________________________________________________
mobilenet_1.00_224 (Model)      (None, 7, 7, 1024)   3228864     tf_op_layer_sub_6[0][0]          
_______________________________________________________________

In [25]:
initial_learning_rate = 5e-3
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
)


model.compile(
    loss="categorical_crossentropy",
    optimizer=keras.optimizers.RMSprop(learning_rate=initial_learning_rate),
    metrics=["accuracy"],
)

In [28]:
checkpoint_cb = keras.callbacks.ModelCheckpoint(
    f"{model_name}.h5", save_best_only=True
)

early_stopping_cb = keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=8)

clr_triangular = CyclicLR(
        mode='triangular2',
        base_lr=1e-8,
        max_lr=1e-3,
        step_size=45
)

history = model.fit(
            train_generator,
            epochs=20,
            steps_per_epoch=197,
            validation_data=val_generator,
            validation_steps=58,
            shuffle=False,
            callbacks=[checkpoint_cb, clr_triangular]
)

Train for 197 steps, validate for 58 steps
Epoch 1/20
197/197 [==============================] - 39s 197ms/step - loss: 1.4589 - accuracy: 0.4337 - val_loss: 3.6621 - val_accuracy: 0.4502
Epoch 2/20
197/197 [==============================] - 53s 270ms/step - loss: 1.3482 - accuracy: 0.4803 - val_loss: 1.5597 - val_accuracy: 0.4502
Epoch 3/20
197/197 [==============================] - 53s 270ms/step - loss: 1.3167 - accuracy: 0.4860 - val_loss: 1.4599 - val_accuracy: 0.4502
Epoch 4/20
197/197 [==============================] - 53s 267ms/step - loss: 1.3152 - accuracy: 0.4816 - val_loss: 1.4666 - val_accuracy: 0.4502
Epoch 5/20
197/197 [==============================] - 53s 268ms/step - loss: 1.2936 - accuracy: 0.4937 - val_loss: 1.4711 - val_accuracy: 0.4502
Epoch 6/20
197/197 [==============================] - 53s 267ms/step - loss: 1.2958 - accuracy: 0.4886 - val_loss: 1.4720 - val_accuracy: 0.4502
Epoch 7/20
197/197 [==============================] - 53s 267ms/step - loss: 1.3062 - a

In [12]:
plt.plot(history.history['val_accuracy'])

NameError: name 'plt' is not defined

In [ ]:
np.max(history.history['val_accuracy'])

In [ ]:
model_load = keras.models.load_model('NASNet_Transfer_Mask_2.h5')

In [ ]:
model_load.evaluate(val_generator)

In [ ]:
with open(f'{model_name}_history.pkl', 'wb') as fh:
    pickle.dump(history.history, fh)